In [1]:


from preprocess_scripts.data_loader import TSFDataLoader
dataset_name = 'calls911_benchmarks'
batch_size = 5
input_size = 16
forecast_horizon=7
feature_type='M'
data_loader = TSFDataLoader(dataset_name,batch_size,input_size,forecast_horizon,feature_type)

2023-12-19 09:54:43.652457: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-19 09:54:43.693956: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-19 09:54:43.693979: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-19 09:54:43.695131: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-19 09:54:43.701967: I tensorflow/core/platform/cpu_feature_guar

42 23 23


In [2]:
import pandas as pd
data = pd.read_csv('./datasets/text_data/EMS-MC/'+dataset_name+'.csv')

In [29]:
data

,date,ABINGTON,AMBLER,BRIDGEPORT,BRYN ATHYN,CHELTENHAM,COLLEGEVILLE,CONSHOHOCKEN,DOUGLASS,EAST GREENVILLE,...,UPPER MORELAND,UPPER POTTSGROVE,UPPER PROVIDENCE,UPPER SALFORD,WEST CONSHOHOCKEN,WEST NORRITON,WEST POTTSGROVE,WHITEMARSH,WHITPAIN,WORCESTER
0,2015-12-01,514.0,49.0,43.0,19.0,339.0,43.0,51.0,63.0,18.0,...,276.0,46.0,191.0,11.0,81.0,121.0,32.0,213.0,182.0,64.0
1,2016-01-01,727.0,82.0,71.0,15.0,586.0,58.0,110.0,146.0,31.0,...,460.0,99.0,285.0,34.0,107.0,202.0,75.0,316.0,258.0,95.0
2,2016-02-01,713.0,57.0,60.0,18.0,548.0,89.0,95.0,109.0,27.0,...,389.0,65.0,281.0,42.0,79.0,182.0,83.0,278.0,271.0,77.0
3,2016-03-01,668.0,63.0,59.0,18.0,488.0,52.0,111.0,121.0,13.0,...,338.0,52.0,248.0,19.0,76.0,168.0,52.0,277.0,241.0,85.0
4,2016-04-01,728.0,73.0,53.0,16.0,531.0,52.0,94.0,115.0,29.0,...,372.0,69.0,256.0,33.0,82.0,182.0,48.0,251.0,243.0,110.0
5,2016-05-01,661.0,71.0,87.0,24.0,511.0,56.0,86.0,82.0,13.0,...,428.0,49.0,245.0,37.0,80.0,202.0,70.0,297.0,254.0,92.0
6,2016-06-01,674.0,81.0,66.0,26.0,560.0,31.0,91.0,105.0,22.0,...,425.0,57.0,279.0,27.0,96.0,199.0,39.0,332.0,270.0,94.0
7,2016-07-01,721.0,74.0,55.0,26.0,559.0,50.0,127.0,116.0,34.0,...,428.0,62.0,263.0,17.0,75.0,194.0,59.0,322.0,225.0,96.0
8,2016-08-01,751.0,56.0,74.0,14.0,572.0,57.0,114.0,95.0,28.0,...,409.0,53.0,304.0,46.0,97.0,204.0,73.0,315.0,227.0,76.0
9,2016-09-01,693.0,71.0,59.0,25.0,557.0,80.0,100.0,110.0,23.0,...,407.0,57.0,311.0,31.0,106.0,190.0,54.0,273.0,247.0,115.0


In [3]:
import pandas as pd
pd.read_csv('./datasets/text_data/EMS-MC/weather.csv').dtypes

date                     object
p (mbar)                float64
T (degC)                float64
Tpot (K)                float64
Tdew (degC)             float64
rh (%)                  float64
VPmax (mbar)            float64
VPact (mbar)            float64
VPdef (mbar)            float64
sh (g/kg)               float64
H2OC (mmol/mol)         float64
rho (g/m**3)            float64
wv (m/s)                float64
max. wv (m/s)           float64
wd (deg)                float64
rain (mm)               float64
raining (s)             float64
SWDR (W/m�)             float64
PAR (�mol/m�/s)         float64
max. PAR (�mol/m�/s)    float64
Tlog (degC)             float64
OT                      float64
dtype: object

In [3]:
train_data = data_loader.get_train()
val_data = data_loader.get_val()
test_data = data_loader.get_test()

2023-12-19 09:55:18.518817: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 26848 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0
2023-12-19 09:55:18.519302: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 31136 MB memory:  -> device: 1, name: Tesla V100-PCIE-32GB, pci bus id: 0000:af:00.0, compute capability: 7.0
2023-12-19 09:55:18.519683: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 21471 MB memory:  -> device: 2, name: Quadro RTX 6000, pci bus id: 0000:d8:00.0, compute capability: 7.5


In [4]:
# !pip install pycausalimpact
from causalimpact import CausalImpact

In [5]:
len(data['date'])-forecast_horizon

49

In [8]:
len(data['date'])

56

In [5]:
ci = CausalImpact(data.loc[:,['AMBLER',"BRIDGEPORT", "BRYN ATHYN", "DOUGLASS", "HATBORO", "HATFIELD BORO", 
                   "LOWER FREDERICK", "NEW HANOVER", "NORRISTOWN", "NORTH WALES", "SALFORD", 
                   "SPRINGFIELD", "TRAPPE"]], [0,len(data['date'])-forecast_horizon], [len(data['date'])-forecast_horizon+1,len(data['date'])-1])
# print(ci.summary())

/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score. The list of unsupported keyword arguments passed include: standardize, nseasons. After release 0.14, this will raise.
  warnings.warn(


In [57]:
print(ci.summary(output='report'))


Analysis report {CausalImpact}


During the post-intervention period, the response variable had
an average value of approx. 75.0. In the absence of an
intervention, we would have expected an average response of 72.16.
The 95% interval of this counterfactual prediction is [64.49, 80.15].
Subtracting this prediction from the observed response yields
an estimate of the causal effect the intervention had on the
response variable. This effect is 2.84 with a 95% interval of
[-5.15, 10.51]. For a discussion of the significance of this effect,
see below.


Summing up the individual data points during the post-intervention
period (which can only sometimes be meaningfully interpreted), the
response variable had an overall value of 450.0.
Had the intervention not taken place, we would have expected
a sum of 432.98. The 95% interval of this prediction is [386.92, 480.89].


The above results are given in terms of absolute numbers. In relative
terms, the response variable showed an increase of +3.9

In [6]:
ci.post_data

,AMBLER,BRIDGEPORT,BRYN ATHYN,DOUGLASS,HATBORO,HATFIELD BORO,LOWER FREDERICK,NEW HANOVER,NORRISTOWN,NORTH WALES,SALFORD,SPRINGFIELD,TRAPPE
50,69.0,60.0,15.0,71.0,88.0,24.0,35.0,73.0,669.0,47.0,21.0,256.0,40.0
51,82.0,61.0,18.0,87.0,78.0,21.0,42.0,83.0,642.0,35.0,24.0,271.0,27.0
52,83.0,44.0,10.0,91.0,54.0,22.0,28.0,68.0,556.0,29.0,27.0,286.0,27.0
53,58.0,50.0,14.0,82.0,105.0,28.0,30.0,102.0,643.0,29.0,9.0,219.0,24.0
54,105.0,53.0,29.0,120.0,95.0,29.0,32.0,79.0,744.0,27.0,38.0,296.0,37.0
55,53.0,65.0,24.0,99.0,90.0,29.0,37.0,83.0,654.0,38.0,33.0,275.0,47.0


In [20]:
ci.post_data.iloc[:,0]

50     69.0
51     82.0
52     83.0
53     58.0
54    105.0
55     53.0
Name: AMBLER, dtype: float64

In [42]:
['a']+['b','c']

['a', 'b', 'c']

In [17]:
ci.inferences.loc[(len(data['date'])-forecast_horizon+1):(len(data['date'])-1),'preds']

50    69.862706
51    73.526743
52    66.395093
53    65.732923
54    84.064309
55    73.393712
Name: preds, dtype: float64

In [25]:
import benchmarks
norm_type = 'B'
activation = 'relu'
dropout = 0.05
n_block=2
no_of_series = 62

In [26]:
build_model = getattr(benchmarks, 'tsmixer').build_model
model = build_model(
    input_shape=(input_size, data_loader.n_feature),
    pred_len=forecast_horizon,
    norm_type=norm_type,
    activation=activation,
    dropout=dropout,
    n_block=n_block,
    ff_dim=no_of_series,
    target_slice=data_loader.target_slice,
)

In [31]:
learning_rate = 0.0001
import tensorflow as tf
import numpy as np

In [33]:
def sMAPE(y_true, y_pred):
    smape_values_per_series = []
    smape_values = (np.abs(y_pred - y_true) /
                    (np.abs(y_pred) + np.abs(y_true))) * 2
    smape_values_per_series.append(np.mean(smape_values, axis=1))
    return np.mean(smape_values_per_series)

In [39]:
def sMAPE_tf(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)

    smape_values = tf.abs(y_pred - y_true) / (tf.abs(y_pred) + tf.abs(y_true)) * 2
    smape_per_series = tf.reduce_mean(smape_values, axis=1)
    mean_smape = tf.reduce_mean(smape_per_series)

    return mean_smape  # Convert TensorFlow tensor to NumPy array for compatibility


In [40]:

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mae', metrics=[sMAPE_tf])


In [41]:
model.fit(
        train_data,
        epochs=15,
        validation_data=val_data,
        # callbacks=[checkpoint_callback, early_stop_callback],
    )

Epoch 1/15


2023-12-18 21:34:47.205212: I external/local_xla/xla/service/service.cc:168] XLA service 0xac5a0d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-18 21:34:47.205290: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2023-12-18 21:34:47.205307: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): Tesla V100-PCIE-32GB, Compute Capability 7.0
2023-12-18 21:34:47.205320: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (2): Quadro RTX 6000, Compute Capability 7.5
2023-12-18 21:34:47.222020: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-18 21:34:47.252866: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1702931687.373178   50679 device_compiler.h:186] Co

4/4 [==============================] - 6s 167ms/step - loss: 1.7132 - sMAPE_tf: 1.4848 - val_loss: 1.9445 - val_sMAPE_tf: 1.4606
Epoch 2/15
4/4 [==============================] - 0s 42ms/step - loss: 1.7087 - sMAPE_tf: 1.4829 - val_loss: 1.9101 - val_sMAPE_tf: 1.4525
Epoch 3/15
4/4 [==============================] - 0s 41ms/step - loss: 1.6962 - sMAPE_tf: 1.4765 - val_loss: 1.8834 - val_sMAPE_tf: 1.4536
Epoch 4/15
4/4 [==============================] - 0s 39ms/step - loss: 1.7019 - sMAPE_tf: 1.4776 - val_loss: 1.8575 - val_sMAPE_tf: 1.4570
Epoch 5/15
4/4 [==============================] - 0s 39ms/step - loss: 1.7080 - sMAPE_tf: 1.4856 - val_loss: 1.8361 - val_sMAPE_tf: 1.4570
Epoch 6/15
4/4 [==============================] - 0s 41ms/step - loss: 1.6888 - sMAPE_tf: 1.4811 - val_loss: 1.8165 - val_sMAPE_tf: 1.4580
Epoch 7/15
4/4 [==============================] - 0s 39ms/step - loss: 1.6655 - sMAPE_tf: 1.4740 - val_loss: 1.7999 - val_sMAPE_tf: 1.4532
Epoch 8/15
4/4 [=====================

In [26]:
model.evaluate(train_data)

4/4 [==============================] - 0s 42ms/step - loss: 3.4850 - mae: 1.4568


[3.484956741333008, 1.456807017326355]

In [8]:

test_result = model.evaluate(test_data)

1/1 [==============================] - 0s 91ms/step - loss: 5.2719 - mae: 1.8610


In [9]:
test_data

<_MapDataset element_spec=(TensorSpec(shape=(None, 16, 62), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7, 62), dtype=tf.float32, name=None))>

In [ ]:
len(df)